# Tutorial: Simple Flight Request
The `google_flights_scraper` package can be used to simulate and end-to-end process of selecting the best departure and return flight as determined and listed by Google Flights.

## Set Up

In [1]:
# Import
from datetime import datetime, timedelta

from google_flights_scraper import GoogleFlightsScraper

# Get today's date
today = datetime.today()

## Request Format

Below are the possible parameters when making a request:  

**Required Inputs**  

`departure_code` : Departure airport, can be airport code or city name  
`departure_country` : Departure country, see `data/airport_codes.csv` for naming  
`arrival_code` : Arrival airport, can be airport code or city name  
`arrival_country` : Departure country, see `data/airport_codes.csv` for naming  
`start_date` : Departure date, in `"MM/DD/YYYY"` format  
`end_date` : Return date, in `"MM/DD/YYYY"` format  
`seat_class` : Class of seat to search for. Options are:  
- Domestic (US):
    - Economy (include Basic)
    - Economy (exclude Basic)
    - Premium economy
    - Business
    - First
- International:
    - Economy
    - Premium economy
    - Business
    - First

**Optional Inputs**  

`export` : True or False, indicating whether to export results  
`export_type` : Type of exports, options are: `"json"` or `"csv"`  
`export_path` : Path to export to  

This will return the following info as a `dict`: 

```python
{
"inputs": {
    "departure_airport",
    "departure_country"
    "arrival_airport",
    "arrival_country",
    "departure_date",
    "return_date",
    "seat_class"
},
"departure_flight": {
    "airline",
    "departure_airport",
    "departure_date",
    "departure_time",
    "num_stops",
    "connection_airports",
    "layover_durations",
    "arrival_airport",
    "arrival_date",
    "arrival_time"
    "duration_minutes", # Total duration of travel (in minutes)
    "duration_str", # Duration formatted as readability (X hr X min)
    "carry_on_bags", # Sometimes None, this information doesn't always populate in Google Flights
    "checked_bags" # Sometimes None, this information doesn't always populate in Google Flights
},
"return_flight": {
    "airline",
    "departure_airport",
    "departure_date",
    "departure_time",
    "num_stops",
    "connection_airports",
    "layover_durations",
    "arrival_airport",
    "arrival_date",
    "arrival_time"
    "duration_minutes", # Total duration of travel (in minutes)
    "duration_str", # Duration formatted as readability (X hr X min)
    "carry_on_bags", # Sometimes None, this information doesn't always populate in Google Flights
    "checked_bags" # Sometimes None, this information doesn't always populate in Google Flights
},
"price",
"price_classification", # One of low, typical, or high. Sometimes null if Google Flights does not provide this info for a specific flight.
"price_relativity", # If price_classificaiton is low, how much cheaper this flight is than usual.
"status" # If the scraper ran successfully, or if not, what the error was.
}
```

## Example 1: Domestic Economy

Below is an example request for a Domestic Economy flight leaving from LAX and arriving at one of the New York airports.

Typical wait time is ~20 seconds.

In [2]:
# Initialize
scraper = GoogleFlightsScraper()

# Create Dates
start = (today + timedelta(weeks=4)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=5)).strftime("%m/%d/%Y")

# Run
result = scraper.scrape_flight(
    departure_code="LAX",
    departure_country="United States of America",
    arrival_code="New York",
    arrival_country="United States of America",
    start_date=start,
    end_date=end,
    seat_class="Economy (include Basic)",
)
result

{'inputs': {'departure_airport': 'LAX',
  'departure_country': 'United States of America',
  'arrival_airport': 'New York',
  'arrival_country': 'United States of America',
  'departure_date': '03/05/2026',
  'return_date': '03/12/2026',
  'seat_class': 'Economy (include Basic)'},
 'departure_flight': {'airline': 'Spirit',
  'departure_airport': 'Los Angeles International Airport',
  'departure_date': 'Thursday, March 5',
  'departure_time': '8:20 AM',
  'num_stops': 0,
  'connection_airports': [],
  'layover_durations': [],
  'arrival_airport': 'Newark Liberty International Airport',
  'arrival_date': 'Thursday, March 5',
  'arrival_time': '4:35 PM',
  'duration_minutes': 315,
  'duration_str': '5 hr 15 min',
  'carry_on_bags': 0,
  'checked_bags': 0},
 'return_flight': {'airline': 'Spirit',
  'departure_airport': 'Newark Liberty International Airport',
  'departure_date': 'Thursday, March 12',
  'departure_time': '6:59 PM',
  'num_stops': 0,
  'connection_airports': [],
  'layover_du

This returns a Nonstop flight on Spirit landing at Newark Liberty International Airport. Google Flights tends to prioritize the cheapest flight as the "best" flight, but sometimes will choose more expensive flights if they offer significantly better travel (usually less layover time).

This price is classified as `"typical"` so a `"price_relativity"` of $0 is returned.

## Example 2: International Premium Economy

Below is an example request for a International Premium Economy flight leaving from New York and arriving in Tokyo.

In [9]:
# Initialize
scraper = GoogleFlightsScraper()

# Create Dates
start = (today + timedelta(weeks=8)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=10)).strftime("%m/%d/%Y")

# Run
result = scraper.scrape_flight(
    departure_code="New York",
    departure_country="United States of America",
    arrival_code="Tokyo",
    arrival_country="Japan",
    start_date=start,
    end_date=end,
    seat_class="Premium economy",
)
result

{'inputs': {'departure_airport': 'New York',
  'departure_country': 'United States of America',
  'arrival_airport': 'Tokyo',
  'arrival_country': 'Japan',
  'departure_date': '04/02/2026',
  'return_date': '04/16/2026',
  'seat_class': 'Premium economy'},
 'departure_flight': {'airline': 'Cathay Pacific',
  'departure_airport': 'John F. Kennedy International Airport',
  'departure_date': 'Thursday, April 2',
  'departure_time': '2:55 PM',
  'num_stops': 1,
  'connection_airports': ['Hong Kong International Airport'],
  'layover_durations': ['6 hr 15 min'],
  'arrival_airport': 'Narita International Airport',
  'arrival_date': 'Saturday, April 4',
  'arrival_time': '6:50 AM',
  'duration_minutes': 1615,
  'duration_str': '26 hr 55 min',
  'carry_on_bags': None,
  'checked_bags': None},
 'return_flight': {'airline': 'JAL and Cathay Pacific',
  'departure_airport': 'Narita International Airport',
  'departure_date': 'Thursday, April 16',
  'departure_time': '6:20 PM',
  'num_stops': 1,
 

This returns a flight on Cathay Pacific with a layover in Hong Kong. You can see the details on the layover, including how long it is.

The `'price_classification'` and `'price_relativity'` here are `None`. I've found that Google Flights offers these suggestions less for non-Economy flights.

## Example 3: Domestic Economy (excluding Basic)

Below is an example request for a Domestic Economy (excluding Basic) flight leaving from SFO and arriving at MIA.

Economy (excluding Basic) excludes budget airlines like Frontier, Spirit, and Breeze from its listings.

In [6]:
# Initialize
scraper = GoogleFlightsScraper()

# Create Dates
start = (today + timedelta(weeks=6)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=7)).strftime("%m/%d/%Y")

# Run
result = scraper.scrape_flight(
    departure_code="SFO",
    departure_country="United States of America",
    arrival_code="MIA",
    arrival_country="United States of America",
    start_date=start,
    end_date=end,
    seat_class="Economy (exclude Basic)",
)
result

{'inputs': {'departure_airport': 'SFO',
  'departure_country': 'United States of America',
  'arrival_airport': 'MIA',
  'arrival_country': 'United States of America',
  'departure_date': '03/19/2026',
  'return_date': '03/26/2026',
  'seat_class': 'Economy (exclude Basic)'},
 'departure_flight': {'airline': 'United',
  'departure_airport': 'San Francisco International Airport',
  'departure_date': 'Thursday, March 19',
  'departure_time': '12:43 AM',
  'num_stops': 1,
  'connection_airports': [],
  'layover_durations': [],
  'arrival_airport': 'Miami International Airport',
  'arrival_date': 'Thursday, March 19',
  'arrival_time': '10:55 AM',
  'duration_minutes': 432,
  'duration_str': '7 hr 12 min',
  'carry_on_bags': 1,
  'checked_bags': 0},
 'return_flight': {'airline': 'United',
  'departure_airport': 'Miami International Airport',
  'departure_date': 'Thursday, March 26',
  'departure_time': '7:23 AM',
  'num_stops': 0,
  'connection_airports': [],
  'layover_durations': [],
  '

This returns a Nonstop flight on United. 

As you can see here the `'price_classification'` is 'cheaper' and this returns a `'price_relativity'` of how much cheaper this flight is than usual ($146)